In [ ]:
import pandas as pd
import numpy as np

In [4]:
ejection_df = pd.read_csv('Fighter_Pilot_Ejection_Success.csv')
ejection_df

,Altitude_ft,Airspeed_knots,Ejection_Type,Pilot_Posture,Time_of_Day,Weather,Terrain,System_Malfunction,Aircraft_Altitude,G_Force,Ejection_Success,Injury_Level
0,15332.28,588.11,Zero-Zero,Optimal,Night,Rain,Water,0,Stable,13.57,1,severe
1,9792.44,312.17,Conventional,Optimal,Night,Rain,Land,1,Stable,13.52,1,severe
2,15878.15,829.16,Zero-Zero,Slouched,Night,Clear,Land,0,Stable,9.26,1,severe
3,6787.09,269.77,Zero-Zero,Slouched,Day,Clear,Land,1,Stable,8.57,1,Minor
4,37862.94,702.04,Zero-Zero,Optimal,Day,Rain,Water,0,Stable,11.47,1,severe
...,...,...,...,...,...,...,...,...,...,...,...,...
2995,19823.87,342.01,Zero-Zero,Optimal,Day,Clear,Land,0,Stable,10.26,1,severe
2996,50000.00,422.60,Zero-Zero,Optimal,Day,Clear,Land,0,Stable,6.98,1,Minor
2997,19957.84,354.12,Zero-Zero,Optimal,Night,Clear,Water,0,Stable,10.62,1,severe
2998,7026.66,1130.39,Zero-Zero,Optimal,Night,Rain,Water,0,Stable,12.12,1,severe


Some of the categorical variables need to be converted into Numerical form for our evaluation purpose 

In [ ]:
ejection_df["Ejection_Type"] = ejection_df["Ejection_Type"].map({"Zero-Zero": 0, "Conventional": 1})

ejection_df["Ejection_Type"]

ejection_df["Pilot_Posture"] = ejection_df["Pilot_Posture"].map({"Optimal": 1, "Slouched": 0})

ejection_df["Pilot_Posture"]

Features of X and Y

In [ ]:
X = ejection_df[["Altitude_ft", "Airspeed_knots", "G_Force", "Ejection_Type", "Pilot_Posture"]].values

Y = ejection_df["Ejection_Success"].values.reshape(-1, 1)

mean = np.mean(X, axis=0)
std = np.std(X, axis=0)

np.save("Mean.npy", mean)
np.save("Standard_dev.npy", std)

X = (X - mean) / std

X = np.hstack([np.ones((X.shape[0], 1)), X])

**Helper Functions**

In [ ]:
def sigmoid(z):
  return 1 / (1 + np.exp(-z))

def compute_cost(X, Y, theta):
  # m = len(Y)
  h = sigmoid(X @ theta)

  cost = (-Y * np.log(h) - (1- Y) * np.log(1 - h)).mean()
  return cost 

def gradient_descent(X, Y, theta, alpha, iterations):
  m = len(Y)
  cost_history = []
  
  for _ in range(iterations):
    h = sigmoid(X @ theta)
    gradient = (X.T @ (h - Y)) / m 
    
    theta = alpha * gradient
    cost_history.append(compute_cost(X, Y, theta))
    
    return theta, cost_history

**Model Training**

In [ ]:
theta = np.zeros((X.shape[1], 1))

alpha = 0.1
iterations = 1000

theta, cost_history = gradient_descent(X, Y, theta, alpha, iterations)

np.save("Trained_theta.npy", theta)

**Predictions**

In [ ]:
def get_user_inputs():
    print("\nEnter the Ejection Parameters to test \n")
    altitude = float(input("Altitude(ft): "))
    airspeed = float(input("Airspeed(Knots): "))
    G_Force = float(input("G Force (G): "))
    ejection_type = int(input("Ejection Type (0 for Zero-Zero, 1 for Conventional): "))
    posture = int(input("Pilot Posture (1 for Optimal, 0 for Slouched): "))
    return np.array([1, altitude, airspeed, G_Force, ejection_type, posture])

# Load trained parameters
theta = np.load("Trained_theta.npy")
mean = np.load("Mean.npy")
std = np.load("Standard_dev.npy")

# Prediction loop
while True:
    try:
        user_input = get_user_inputs()
        normalized_input = (user_input[1:] - mean) / std  # Exclude bias term
        normalized_input = np.hstack([1, normalized_input])  # Re-add bias

        probability = sigmoid(normalized_input @ theta)
        prediction = "SUCCESS" if probability >= 0.5 else "FAILURE"
        
        print(f"\nPrediction: {prediction} (Probability: {probability:.2f})")
        print("---")

    except ValueError:
        print("Error: Invalid Input! Please enter proper numerical values.")
    except KeyboardInterrupt:
        print("\nExiting...")
        break